In [46]:
import numpy as np
import pandas as pd

%matplotlib inline

## Params

In [47]:
lgb_submit = pd.read_csv('./data/submission/lgb_estim_validation.csv')
tf_submit = pd.read_csv('./data/submission/tf_estim_validation.csv')
item_ensemble_submit = pd.read_csv('./data/submission/ens_estim_validation.csv')
prophet_top_down_submit = pd.read_csv('./data/submission/Prophet_store_dpt_lgb_weights_validation.csv')
wavenet_top_down_submit = pd.read_csv('./data/submission/Wavenet_store_dpt_lgb_weights_validation.csv')
lgb_top_down_submit = pd.read_csv('./data/submission/Lgb_store_dpt_lgb_weights_validation.csv')

for c in range(1,29):
    tf_submit['F%s' % c].clip(0, inplace=True)

In [48]:
true = pd.read_csv('data/raw/sales_train_evaluation.csv')[['id'] + ['d_%s' % c for c in range(1914, 1942)]]
true.columns = ['id'] + ['F%s' % c for c in range(1, 29)]
true['id'] = true['id'].str.replace('evaluation', 'validation')

In [49]:
import functools

errors = functools.reduce(pd.merge,
                 [
                     pd.melt(lgb_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb'),
                     pd.melt(tf_submit.query('id.str.contains("validation")'), id_vars='id', value_name='tf'),
                     pd.melt(item_ensemble_submit.query('id.str.contains("validation")'), id_vars='id', value_name='item_ensemble'),
                     pd.melt(prophet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='prophet_top_down'),
                     pd.melt(wavenet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='wavenet_top_down'),
                     pd.melt(lgb_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb_top_down'),
                     pd.melt(true, id_vars='id', value_name='true')
                 ]
                )

errors['neutral'] = 0

In [50]:
pd.read_csv('./data/raw/sales_train_validation.csv')

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [52]:
errors['store_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[3:5]))
errors['dept_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
errors['item_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[0:3]))

In [53]:
list_ensemble = list()
weights_dict = {}
for _, gp in errors.groupby(['store_id', 'dept_id']):
    errors_gp = list()
    for m in ['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'neutral']:
        errors_gp.append(np.sqrt(((gp['true']-gp[m])**2).mean()))
    
    es = errors_gp[:6]
    ps = gp[['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values
    e0 = errors_gp[-1]
    l=0.0001
    m = len(es)
    n = len(ps)
    X = ps
    pTy = .5 * (n * e0**2 + (X**2).sum(axis=0) - n * np.array(es)**2)
    w = np.linalg.pinv(X.T.dot(X) + l * n * np.eye(m)).dot(pTy)
    weights_dict[_] = w
    
    gp['ensemble'] = gp[['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values.dot(w)
    list_ensemble.append(gp)

/home/dataexmachina/DataProjects/m5-forts-castors/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
ensemble = pd.concat(list_ensemble)
for m in ['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'ensemble']:
    print(m, np.sqrt(((ensemble['true']-ensemble[m])**2).mean()))

lgb 2.0887245479323964
tf 2.0980360379853136
prophet_top_down 2.113325014023945
wavenet_top_down 2.118072537261426
lgb_top_down 2.1268339606220357
ensemble 2.0569213361619516


In [55]:
ensemble['ensemble'].clip(0, inplace=True)

In [56]:
ensemble = ensemble.pivot_table(index=['id'], columns=['variable'], values=['ensemble']).reset_index()
ensemble.columns = [c[0] if c[1]=='' else c[1] for c in ensemble.columns.tolist()]
ensemble = ensemble[['id'] + ['F%s' % c for c in range(1,29)]]

In [57]:
ensemble

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.857764,0.809376,0.787843,0.762471,1.079831,0.957376,0.951140,0.825664,0.859507,...,0.990731,1.097793,1.029880,0.756797,0.759406,0.711665,0.734538,0.933971,0.962977,0.950181
1,FOODS_1_001_CA_2_validation,1.094436,1.127991,1.138655,1.440572,1.672516,1.449939,1.640522,1.035464,1.050082,...,1.437858,1.544395,1.712673,1.150365,1.063161,1.044738,1.072222,1.352111,1.713995,1.465642
2,FOODS_1_001_CA_3_validation,1.343599,1.384312,1.334332,1.380034,1.588636,1.203238,1.056957,1.236676,1.326210,...,1.441638,1.352179,1.808497,1.180660,1.182189,1.113614,1.175600,1.356365,1.159212,1.101248
3,FOODS_1_001_CA_4_validation,0.439887,0.450647,0.489580,0.472735,0.521932,0.504955,0.580150,0.442844,0.462531,...,0.544648,0.480875,0.504019,0.433803,0.451287,0.468185,0.440755,0.475369,0.446723,0.501569
4,FOODS_1_001_TX_1_validation,0.152444,0.182918,0.167624,0.140728,0.135566,0.009651,0.153915,0.590767,0.470371,...,0.347834,0.336442,0.528590,0.362623,0.342182,0.331543,0.277864,0.303828,0.321121,0.369850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_validation,0.240157,0.202863,0.233445,0.228862,0.273592,0.354659,0.304701,0.227992,0.224677,...,0.262931,0.331110,0.314220,0.227820,0.198107,0.216809,0.216765,0.263757,0.305052,0.299920
30486,HOUSEHOLD_2_516_TX_3_validation,0.188974,0.185694,0.169063,0.178594,0.197720,0.230199,0.172839,0.164617,0.156046,...,0.168270,0.194809,0.183783,0.137016,0.153492,0.145130,0.156885,0.174021,0.197459,0.183969
30487,HOUSEHOLD_2_516_WI_1_validation,0.073435,0.068751,0.069819,0.075982,0.100346,0.103571,0.082459,0.080070,0.080900,...,0.105742,0.127094,0.105133,0.080013,0.077425,0.078711,0.084930,0.110166,0.136030,0.113782
30488,HOUSEHOLD_2_516_WI_2_validation,0.056728,0.053464,0.024891,0.061789,0.065498,0.132026,0.078579,0.056576,0.071620,...,0.041616,0.095480,0.050732,0.050075,0.087028,0.061644,0.057487,0.063572,0.089014,0.074471


In [58]:
ensemble.to_csv('data/submission/netflix_ensemble.csv', index=False)